In [1]:
import numpy as np
import pandas as pd

In [2]:
from typing import List, Dict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
songs=pd.read_csv('songdata.csv')
songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
songs=songs.sample(n=5000).drop('link',axis=1).reset_index(drop=True)

In [5]:
songs['text'] = songs['text'].str.replace(r'\n','')

<ipython-input-5-59ee86f53df0>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  songs['text'] = songs['text'].str.replace(r'\n','')


In [6]:
tfidf = TfidfVectorizer(analyzer='word',stop_words='english')

In [7]:
lyrics_matrix=tfidf.fit_transform(songs['text'])

In [9]:
cosine_similarities = cosine_similarity(lyrics_matrix)
similarities= {}

In [10]:
for i in range(len(cosine_similarities)):
    #Sort each element in cosine_similarities and get the indexes of the songs, #store in similarities each name of the 50 most similar song,
    #except the first one that is the same song.
    similar_indices=cosine_similarities[i].argsort()[:-50:-1] 
    similarities[songs['song'].iloc[i]]=[(cosine_similarities[i][x],songs['song'][x],songs['artist'][x]) for x in similar_indices][1:]

In [11]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)
        
        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score") 
        
    def recommend(self, recommendation):
        song=recommendation['song']
        #Get number of songs to recommend
        number_songs=recommendation['number_songs']
        #Get the number of songs most similars from matrix similarities
        recom_song=self.matrix_similar[song][:number_songs]
        self._print_message(song=song,recom_song=recom_song)

In [12]:
recommedations=ContentBasedRecommender(similarities)

In [13]:
recommendation = {
    "song": songs['song'].iloc[10],
    "number_songs": 4 
}

songs['song'].iloc[10]

'Turn'

In [14]:
recommedations.recommend(recommendation)

The 4 recommended songs for Turn are:
Number 1:
The Other Half (Of Me) by Within Temptation with 0.416 similarity score
Number 2:
I Want To Know What Love Is by Foreigner with 0.295 similarity score
Number 3:
For This Cause by Hillsong with 0.286 similarity score
Number 4:
Don't Want To Know by John Martyn with 0.279 similarity score


In [15]:
recommendation2 = {
    "song": songs['song'].iloc[120],
    "number_songs": 4 
}
songs['song'].iloc[120]

'Candy Kisses'

In [16]:
recommedations.recommend(recommendation2)

The 4 recommended songs for Candy Kisses are:
Number 1:
Mean Mean Man by Wanda Jackson with 0.272 similarity score
Number 2:
THERE'S NOTHING AS SWEET AS MY BABY by Hank Williams with 0.214 similarity score
Number 3:
One More Kiss Dear by Vangelis with 0.176 similarity score
Number 4:
Nothing by Fastball with 0.172 similarity score
